In [1]:
!pip install numpy==2.0.0 mediapipe==0.10.14 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 18.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.


In [2]:
import cv2
import os
import mediapipe as mp
import math
import numpy as np

In [3]:
!unrar x Training.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from Training.rar

Creating    Training                                                  OK
Extracting  Training/001.tif                                               0%  1%  OK 
Extracting  Training/017.tif                                               1%  2%  OK 
Extracting  Training/043.tif                                               2%  3%  4%  OK 
Extracting  Training/044.tif                                               4%  5%  OK 
Extracting  Training/055.tif                                               5%  6%  7%  OK 
Extracting  Training/067.tif                                               7%  8%  OK 
Extracting  Training/070.tif                                               8%  9% 10%  OK 
Extracting  Training/078.tif                                              10% 11%  O

In [4]:
IMAGE_DIR = 'Training'
OUTPUT_DIR = 'output'
THRESHOLD = 0.98

In [5]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [6]:
def cosine_similarity(v1, v2):
    dot = v1[0] * v2[0] + v1[1] * v2[1]
    norm1 = math.hypot(v1[0], v1[1])
    norm2 = math.hypot(v2[0], v2[1])
    if norm1 == 0 or norm2 == 0:
        return 0
    return dot / (norm1 * norm2)

In [10]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5
)
mp_draw = mp.solutions.drawing_utils

In [11]:
for filename in os.listdir(IMAGE_DIR):
    if not filename.lower().endswith('.tif'):
        continue
    img_path = os.path.join(IMAGE_DIR, filename)
    image = cv2.imread(img_path)
    h, w, _ = image.shape
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    result = hands.process(rgb)
    if not result.multi_hand_landmarks:
        print(f'No hand detected in {filename}')
        continue
    hand_lms = result.multi_hand_landmarks[0]

    lm_coords = []
    for lm in hand_lms.landmark:
        lm_coords.append((int(lm.x * w), int(lm.y * h)))

    tip_ids = [4, 8, 12, 16, 20]
    base_ids = [2, 5, 9, 13, 17]
    tips = [lm_coords[i] for i in tip_ids]
    valleys = []
    for j in range(4):
        i = base_ids[j]
        i_next = base_ids[j + 1]
        x_coord = (lm_coords[i][0] + lm_coords[i_next][0]) / 2
        y_coord = (lm_coords[i][1] + lm_coords[i_next][1]) / 2
        valleys.append((x_coord, y_coord))

    pose_parts = []
    for i, tip in enumerate(tips[:-1]):
        nxt = tips[i+1]
        dist = cosine_similarity(tip, nxt)
        pose_parts.append('+' if dist < THRESHOLD else '-')
    pose_code = ''.join(f'{i+1}{pose_parts[i]}' for i in range(len(pose_parts)))
    pose_code += '5'

    res_path = os.path.join(OUTPUT_DIR, f'{os.path.splitext(filename)[0]}_Results.txt')
    with open(res_path, 'w') as f:
        f.write(pose_code + '\n')
        parts = [f'!,{filename}']
        for x, y in tips:
            parts.append(f'T {x} {y}')
        for x, y in valleys:
            parts.append(f'V {x} {y}')
        parts.append('?')
        f.write(','.join(parts) + '\n')

    cv2.putText(image, pose_code, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    line_pts = []
    for i in range(len(tips)):
        line_pts.append(tips[i])
        if i < len(valleys):
            line_pts.append(valleys[i])
    cv2.polylines(image, [np.array(line_pts, dtype=np.int32)], False, (255, 0, 0), 2)

    out_img_path = os.path.join(OUTPUT_DIR, filename)
    cv2.imwrite(out_img_path, image)
    print(f'Processed {filename}: pose={pose_code}')

hands.close()

/usr/local/lib/python3.11/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processed 209.tif: pose=1+2-3-4-5
Processed 134.tif: pose=1+2-3-4-5
Processed 043.tif: pose=1+2-3-4-5
Processed 295.tif: pose=1-2+3+4+5
Processed 239.tif: pose=1+2-3+4+5
Processed 001.tif: pose=1-2-3-4+5
Processed 251.tif: pose=1+2-3-4+5
Processed 091.tif: pose=1+2-3-4-5
Processed 182.tif: pose=1+2-3-4-5
Processed 152.tif: pose=1-2-3+4-5
Processed 292.tif: pose=1-2+3-4-5
Processed 300.tif: pose=1+2-3-4+5
Processed 146.tif: pose=1-2-3+4+5
Processed 267.tif: pose=1-2-3-4+5
Processed 265.tif: pose=1-2+3+4-5
Processed 175.tif: pose=1+2-3-4+5
Processed 231.tif: pose=1+2-3-4+5
Processed 230.tif: pose=1+2-3-4+5
Processed 144.tif: pose=1+2-3-4+5
Processed 225.tif: pose=1+2-3-4+5
Processed 158.tif: pose=1+2-3-4+5
Processed 274.tif: pose=1-2-3-4+5
Processed 240.tif: pose=1+2-3-4+5
Processed 201.tif: pose=1+2-3-4-5
Processed 067.tif: pose=1-2-3-4-5
Processed 281.tif: pose=1+2-3-4-5
Processed 017.tif: pose=1+2-3-4-5
Processed 107.tif: pose=1-2-3+4+5
Processed 284.tif: pose=1-2-3-4+5
Processed 078.